In [42]:
import pandas

In [2]:
apartments = pandas.read_csv('rent_all.csv')
apartments.head()

,id,building,street,district,room,rent,rentk,area,lower,year,month
0,1,SKYLINE RESIDENCES,TELOK BLANGAH ROAD,4,3,7300,7.3,1300 to 1400,1300,22,12
1,2,REFLECTIONS AT KEPPEL BAY,KEPPEL BAY VIEW,4,3,5800,5.8,1500 to 1600,1500,22,12
2,3,ONE SHENTON,SHENTON WAY,1,2,7300,7.3,1100 to 1200,1100,22,12
3,4,THE ROCHESTER RESIDENCES,ROCHESTER DRIVE,5,2,6500,6.5,1300 to 1400,1300,22,12
4,5,THE RESIDENCES AT W SINGAPORE SENTOSA COVE,OCEAN WAY,4,2,7000,7.0,1200 to 1300,1200,22,12


In [3]:
apartments["room"] = apartments.groupby("building").cumcount() + 1
apartments = apartments[["id", "building", "rent", "room"]]
apartments.head()

,id,building,rent,room
0,1,SKYLINE RESIDENCES,7300,1
1,2,REFLECTIONS AT KEPPEL BAY,5800,1
2,3,ONE SHENTON,7300,1
3,4,THE ROCHESTER RESIDENCES,6500,1
4,5,THE RESIDENCES AT W SINGAPORE SENTOSA COVE,7000,1


In [4]:
max_room_per_building = apartments.groupby('building')['room'].max().reset_index()
apartments = apartments[apartments['building'].isin(
    max_room_per_building[max_room_per_building['room'] <= 2000]['building']
)]

In [5]:
apartments = apartments[~apartments["building"].isin(['# 1 SUITES', '# 1 LOFT'])]

In [6]:
apartments = apartments[apartments["building"].isin([
    x for x in apartments["building"].unique()
        if "CONDOMINIUM" in x or "BUILDING" in x or "RESIDENCES" in x])]

In [7]:
apartments.to_csv('apartments.csv', index=False)

In [8]:
debtors = pandas.read_csv('application_data.csv')
debtors.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
columns = [
    'SK_ID_CURR',
    'TARGET',
    'CODE_GENDER',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY',
    'AMT_INCOME_TOTAL',
    'CNT_CHILDREN',
    'AMT_CREDIT',
    'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'FLAG_MOBIL',
    'FLAG_PHONE',
    'FLAG_EMAIL',
    'OCCUPATION_TYPE',
    'ORGANIZATION_TYPE'
]
debtors = debtors[columns]
debtors.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,CNT_CHILDREN,AMT_CREDIT,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,FLAG_MOBIL,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,ORGANIZATION_TYPE
0,100002,1,M,N,Y,202500.0,0,406597.5,Working,Secondary / secondary special,Single / not married,1,1,0,Laborers,Business Entity Type 3
1,100003,0,F,N,N,270000.0,0,1293502.5,State servant,Higher education,Married,1,1,0,Core staff,School
2,100004,0,M,Y,Y,67500.0,0,135000.0,Working,Secondary / secondary special,Single / not married,1,1,0,Laborers,Government
3,100006,0,F,N,Y,135000.0,0,312682.5,Working,Secondary / secondary special,Civil marriage,1,0,0,Laborers,Business Entity Type 3
4,100007,0,M,N,Y,121500.0,0,513000.0,Working,Secondary / secondary special,Single / not married,1,0,0,Core staff,Religion


In [4]:
def search_and_set_apartment(value):
    global last_index
    current_apartment = None
    minimum = sorted_apartments[last_index:]["rent"].min()
    
    new_value = value / 24
    index = last_index
    for _, row in sorted_apartments[last_index:].iterrows():
        difference = abs(new_value - row['rent'])
        if minimum < difference:
            minimum = difference
            index += 1
            continue
        
        index += 1
        last_index = index
        current_apartment = row['id']
        break
    
    return current_apartment

last_index = 0
sorted_apartments = apartments.sort_values(by=['rent'])
sorted_debtors = debtors.sort_values(by="AMT_CREDIT")

max_debit = sorted_debtors['AMT_CREDIT'].max() / 24
filtered_debtors = sorted_debtors[sorted_debtors['AMT_CREDIT'] < max_debit]
filtered_debtors["APARTMENT_ID"] = filtered_debtors['AMT_CREDIT'].apply(
    search_and_set_apartment)
debtors = filtered_debtors

In [1]:
from faker import Faker
faker = Faker('pt_BR')

In [97]:
def generate_name(gender):
    if gender == "M":
        return faker.name_male()
    return faker.name_female()

debtors["SK_ID_CURR"] = debtors["SK_ID_CURR"].apply(lambda x: faker.cpf())
debtors["NAME"] = debtors["CODE_GENDER"].apply(generate_name)

In [99]:
from datetime import datetime
date_func = lambda x: faker.date_between(start_date='-5y', end_date='today')
vectorize = numpy.vectorize(date_func, otypes=[datetime])
due_date = numpy.empty(len(debtors), dtype=datetime)
due_date[:] = vectorize(due_date[:])

In [100]:
import numpy, random

installments = numpy.random.randint(1, 24, size=len(debtors))

invoices = pandas.DataFrame({ 
    "INVOICE_ID": range(1, len(debtors) + 1),
    "TENANT_CPF": debtors["SK_ID_CURR"],
    "AMOUNT": debtors["AMT_CREDIT"] / installments,
    "DUE_DATE": due_date,
    "PAYMENT_DATE": due_date
})

In [101]:
from datetime import timedelta

days_func = lambda x: timedelta(days=random.randint(0, 30))
vectorize_days = numpy.vectorize(days_func, otypes=[timedelta])

payment_after = invoices["PAYMENT_DATE"].apply(
    lambda x: faker.date_between(start_date=x, end_date='today')
)
not_paid = payment_after == invoices["DUE_DATE"]
late_payment = numpy.where(not_paid, None, payment_after),

payment_before = vectorize_days(invoices["PAYMENT_DATE"])
payment_on_day = invoices["PAYMENT_DATE"] - payment_before

invoices["PAYMENT_DATE"] = numpy.where(
    debtors["TARGET"], late_payment, payment_on_day
)[0]

In [141]:
invoices_amount = debtors["AMT_CREDIT"] / invoices["AMOUNT"]
current_date = datetime.now().date()

for index, tenant in invoices.iterrows():
    last_id = len(invoices)
    late_date_installments = invoices_amount[index]

    last_date = tenant["DUE_DATE"]
    months = int((current_date - last_date).days / 31)

    is_a_good_tenant = True
    if (tenant["PAYMENT_DATE"] is None or
        tenant["PAYMENT_DATE"] > tenant["DUE_DATE"]):
        is_a_good_tenant = False
    
    due_date_array = []
    payment_date_array = []
    for i in range(months):
        last_date = last_date + timedelta(days=31)
        if is_a_good_tenant or late_date_installments == 0:
            payment_date = last_date - timedelta(days=random.randint(0, 30))
        else:
            payment_date = faker.date_between(start_date=last_date,
                                              end_date='today')
            if payment_date == last_date: payment_date = None
        due_date_array.append(last_date)
        payment_date_array.append(payment_date)

    new_invoices = pandas.DataFrame({ 
        "INVOICE_ID": range(last_id, last_id + months),
        "TENANT_CPF": numpy.full(months, tenant["TENANT_CPF"]),
        "AMOUNT": numpy.full(months, tenant["AMOUNT"]),
        "DUE_DATE": due_date_array,
        "PAYMENT_DATE": payment_date_array
    })
    invoices = pandas.concat([invoices, new_invoices], ignore_index=True)

In [146]:
debtors.to_csv('debtors.csv', index=False)
invoices.to_csv('invoices.csv', index=False)
apartments.to_json("apartments.json", orient="records")

In [2]:
import sqlite3, pandas

invoices = pandas.read_csv("invoices.csv")
connection = sqlite3.connect("reco.db")
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS invoices (
    invoice_id INTEGER PRIMARY KEY,
    tenant_cpf TEXT NOT NULL,
    amount REAL NOT NULL,
    due_date TEXT NOT NULL,
    payment_date TEXT
);
""")

invoices.to_sql("INVOICES", connection, if_exists="replace", index=False)
               
connection.commit()